In [1]:
import svgling
from svgling.figure import SideBySide, RowByRow, Caption
# two sample trees
t0 = ("S", ("NP", "D", "N"), ("VP", "V", ("NP", "D", "A", "N")))
t1 = ("S", ("NP", ("D", "the"), ("N", "rhinoceros")), ("VP", ("V", "saw"), ("NP", ("D", "the"), ("A", "gray"), ("N", "elephant"))))
demo_trees = [t0, t1]

# Overview of `svgling`

The `svgling` package is a python tree-drawing package, aimed at linguists who want to draw constituent trees.

* [1. Interface](#1.-Interface)
* [2. Layout and display options](#2.-Layout-and-display-options)
   - [Layout overview](#Layout-overview)
   - [Debug mode](#Debug-mode)
   - [Overall layout options](#Overall-layout-options)
   - [Line and node positioning options](#Line-and-node-positioning-options)
   - [Text options](#Text-options)
* [3. Complex figures](#3.-Complex-figures)

## 1. Interface

The main interface to `svgling` is `svgling.draw_tree`. This function takes a tree description, and named arguments specifying options. The valid options are described below, and options parameters are used to construct a `svgling.core.TreeOptions` object; such an object can be passed directly and passed via a named argument `options`. The default options can be accessed via `svgling.core.TreeOptions()`.

In [11]:
svgling.draw_tree(t0)

The tree description can be in one of two forms: an indexable object (e.g. list, tuple) consisting of a head at index 0 and a possibly empty sequence of subtrees at indices $1..n$, or an object implementing the `nltk.Tree` api, which stores the head label in the function `.label()` and the daughter subtrees as indices on the object. See `t0` and `t1` above for examples of the indexable format, but this is pretty standard notation going back to lisp.

The package also will monkey-patch `nltk` if it's available, adding the function `nltk.Tree._repr_svg_()` that calls `draw_tree(self)` behind the scenes. To adjust options for nltk trees, you can set values on `svgling.core.nltk_tree_options`, which is a `nltk.core.TreeOptions` instance. If you want to go further, `svgling.disable_nltk_png()` will completely remove the default `nltk` tree-drawing code, preventing it from ever being called. (Jupyter will typically try every `_repr_*_` call and store the results in a notebook file, even though it has a preference which to show.)

### Nodes

A tree node is (for now) just a string. This string may span multiple lines, which are separated by `\n`. For example: `"N\ncat"` gives a two-line node with `N` as the first like and `cat` as the second. Within a node, multiple lines are anchored at the middle (leading to centering). A label consisting of the empty string `""` is considered empty, and will not render at all -- the lines will be joined in the middle of the row height. If you want a blank label, any sequence of whitespace will work, e.g. `" "`.

## 2. Layout and display options

The customizable `TreeOptions` parameters are described in the rest of this section.

### Layout overview

**Vertical layout**: A node at depth $n$ (where the root node of the tree is depth $0$) is positioned vertically in a line with all other nodes of depth $n$. Exception (see below for examples): if `leaf_nodes_align` is set to true, any leaf nodes are aligned with the lowest level of the tree, rather than depth they would otherwise be at. Within a row, vertical space is allowed for the tallest node at that depth; positioning of shorter nodes at that depth is configurable. Vertical spacing is calculated/generated in *em*s.

**Horizontal layout**: The horizontal position of daughter nodes relative to a parent is determined by a (configurable) algorithm, usually based on some measure of the size of the daughter nodes. By default, this algorithm estimates the max text width taken up by the parent node label or the width of daughter nodes (and their daughters, etc). See below for examples of other options. No node will be positioned vertically below a node that does not dominate it. There is also a configurable padding parameter. Horizontal spacing is calculated initially in estimated *em*s, but (canvas width aside) is converted to percentages for svg layout. Because `svgling` does not do multi-pass rendering, it uses heuristics for glyph width rather than accurately calculating glyph width. (To do this, you'd basically need to render to a device, and see what happens.)

**Canvas layout**: The canvas width is estimated from node text width + padding. The canvas height is determined by the tree depth, level heights, with an extra 1em at the bottom of the canvas for descenders from leaf node glyphs.

### Debug mode

**`debug`**: When this option is set to `True`, the rendered will show a `1em` grid, as well as a red box for every subtree. This can be useful if something isn't doing what you expect. Several of the documentation examples below use this to illustrate various spacing options.

In [2]:
svgling.draw_tree(t1, debug=True)

### Overall layout options

The following are `TreeOptions` parameters that affect layout.

**`horiz_spacing`**: This parameter determines how daughter nodes are spaced horizontally relative to the parent. Possible values are `svgling.core.HorizSpacing.TEXT` (default; space proportionally based on estimated text width), `svgling.core.HorizSpacing.EVEN` (space evenly based on number of immediate daughters), and `svgling.core.HorizSpacing.NODES` (space proportionally to the number of leaf nodes in the subtrees).

Usually `TEXT` looks best, but the other two may be preferable for abstract trees where label widths are all similar. Without manual adjustment, the two other options will deal poorly with long labels.

In [3]:
examples = list()
for t in range(len(demo_trees)):
    row = list()
    for opt in svgling.core.HorizSpacing:
        # debug mode on so that the exact rendering differences are very obvious
        row.append(Caption(svgling.draw_tree(demo_trees[t], horiz_spacing=opt, debug=True), "Example t%d with horiz_spacing=%s" % (t, str(opt))))
    examples.append(SideBySide(*row))

RowByRow(*examples)

**`average_glyph_width`**: A heuristic factor used to calculate text widths; basically, a divisor in *em*s. Defaults to `2.0`. Does not generally need to be adjusted for default settings (which try to just use Times), but may be worth adjusting for custom fonts.

**`leaf_padding`**: An amount to pad each leaf by, in glyphs. Will be divided by `average_glyph_width`. Default is 2. Negative values are possible, but will usually result in text being cut off. Leaf padding is applied as a constant to the overall canvas size regardless of the value of `horiz_spacing` (i.e. the canvas size is always determined by the sum of node widths plus leaf padding at every widest subtree), so will impact spacing to some degree for any setting of this option, but is only applied directly to each leaf for `HorizSpacing.TEXT`.

In [12]:
examples = list()
for i in (0, 2, 5, -2):
    examples.append(Caption(svgling.draw_tree(t1, leaf_padding=i), "Example t1 with leaf_padding=%g" % i))
SideBySide(*examples)

**`vert_align`**: How row alignment when there are multi-line labels should be calculated. If all labels in a row have the same height, this has no impact, but if there are differences, it controls the position of the shorter node labels vertically. Default is centered. For empty labels, this affects the position of the line join. The values `TOP`, `CENTER`, and `BOTTOM` are self-explanatory. `svgling.VertAlign.EVEN` causes all nodes to be treated as the same height (relative to their row) regardless of text contents, even empty nodes.

In [5]:
t2 = ("X", ("multiline\nlabel", "Y"), ("", "Z\nA"), ("A", "B"), "C")

examples = list()
for opt in svgling.core.VertAlign:
    examples.append(Caption(svgling.draw_tree(t2, vert_align=opt), "Example t2 with vert_align=%s" % str(opt)))

SideBySide(*examples)

**`distance_to_daughter`**: The distance between rows in *em*s -- that is, distance from the bottom of one row to the top of another. Values less than about 0.5 are not recommended and will usually result in rendering oddities. Note that line starts are 0.2*em*s below a node label, so 0.2 will give completely horizontal lines (not 0.0).

Default is 2.

In [6]:
examples = list()
for i in (0.5, 2, 4, 0.2):
    examples.append(Caption(svgling.draw_tree(t1, distance_to_daughter=i), "Example t1 with distance_to_daughter=%g" % i))
SideBySide(*examples)

### Line and node positioning options

**`leaf_nodes_align`**: if true, will align all leaf nodes with the lowest depth leaf nodes in the tree.

In [7]:
svgling.draw_tree(t1, leaf_nodes_align=True)

**`descend_direct`**: When an edge skips levels (currently only possible for leaf nodes, when `leaf_nodes_align=True`), should the line go directly from the parent to the daughter? If `False`, the line will go to the position that the daughter would have been at as if there is an empty node there, and descend vertically. This can be useful for very deep trees where a `True` value results in overlapping, and also just look better. However, it doesn't allow distinguishing empty nodes visually in the tree. As with empty nodes, the positioning of the line join is affected by `vert_align`. Defaults to `True`.

The following example shows a tree that renders quite badly without this option set to `True`, because of the asymmetry between leaf node widths.

In [8]:
t3 = ("A", "B", ("C", ("D", "middle leaf"), "H"), ("E", "long leaf", "G"))
examples = list()
for opt in (True, False):
    examples.append(Caption(svgling.draw_tree(t3, leaf_nodes_align=True, descend_direct=opt), "Example t3 with descend_direct=%s" % str(opt)))
SideBySide(*examples)

### Text options

**`global_font_style`**: a css-formatted string that will be used to style text in the tree. Since this is css, you can put all sorts of stuff in it, but I recommend at least including `font-family`, `font-weight`, and `font-style`, because without these font rendering may be inconsistent depending on where the svg is embedded (for example showing as serif in some settings, sans-serif in others). If you are sharing your svg with others, I recommend sticking to web-safe fonts. The default values is: `"font-family: times, serif; font-weight:normal; font-style: normal; font-size: 12pt;"`.

In [13]:
styles = ("font-family: georgia, times, serif; font-weight:normal; font-style: normal; font-size: 20pt",
          "font-family: sans-serif; font-weight:normal; font-style: normal; font-size: 12pt;",
          "font-family: impact, times, serif; font-weight:normal; font-style: italic; font-size: 10pt")

examples = list()
for i in range(len(styles)):
    examples.append(Caption(svgling.draw_tree(t1, global_font_style=styles[i]), "Example t1 with style %d" % i))
SideBySide(*examples) # n.b. SideBySide is kind of brittle with font changes...

## 3. Complex figures

As several of the above examples illustrate, `svgling.figure` provides facilities for generating more complex figures out of svg drawings. These classes are still rather basic, and they don't do well for really arbitrary figures, but they do well enough for combining `svgling` trees together. In principle, they work on any object that implements the following functions:

 * `get_svg()`: return an `svgwrite` object.
 * `height()`: get the diagram height *in ems*.
 * `width()`: get the diagram width *in ems*.

All the classes described here implement this interface, and so may be combined.

As you might guess, the dependence on *em*s imposes some limitations and complexities when trying to combine subfigures that have different styling.

**`Caption`**: This places a text caption below an svg drawing.

**`SideBySide`**: This places a list of svg drawings in a row.

**`RowByRow`**: This places a list of svg drawings in a column.